# Lab 37: AI-Powered Threat Actors

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab37_ai_powered_threats.ipynb)

Understanding how adversaries leverage AI in modern attacks and building defenses.

## Learning Objectives

1. Understand AI attack vectors used by threat actors
2. Detect AI-generated phishing content
3. Analyze AI-enhanced malware patterns
4. Build defenses against AI-powered attacks

> **Responsible Use**: This lab is for defensive education only.

**Next:** Lab 17 (Adversarial ML) or Lab 17a (ML Security Intro primer)

In [ ]:
# Standard imports
import re
import math
from dataclasses import dataclass
from typing import List, Optional
from enum import Enum

## Part 1: AI-Powered Threat Landscape

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    AI-POWERED THREAT LANDSCAPE                              │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   RECONNAISSANCE          WEAPONIZATION           DELIVERY                  │
│   ┌─────────────┐        ┌─────────────┐        ┌─────────────┐            │
│   │ AI-Powered  │        │ AI-Generated│        │ Deepfake    │            │
│   │ OSINT       │───────>│ Malware     │───────>│ Phishing    │            │
│   │ Scraping    │        │ Variants    │        │ Vishing     │            │
│   └─────────────┘        └─────────────┘        └─────────────┘            │
│                                                                             │
│   DEFENSE IMPLICATIONS:                                                     │
│   • Traditional signatures fail against AI-generated variants               │
│   • Human-based detection struggles with deepfakes                          │
│   • Volume of attacks increases exponentially                               │
└─────────────────────────────────────────────────────────────────────────────┘
```

## Part 2: AI-Generated Phishing Detection

AI-generated phishing differs from traditional phishing:
- Unusually consistent quality across messages
- Lack of typical human typos/mistakes
- Generic personalization patterns
- Statistically "too perfect" language

In [ ]:
@dataclass
class PhishingAnalysis:
    """Results of AI-generated content analysis."""
    text: str
    ai_probability: float
    indicators: List[str]
    confidence: str
    recommendations: List[str]


class AIPhishingDetector:
    """Detect AI-generated phishing content."""
    
    def __init__(self):
        # Common AI-generated phrases
        self.ai_phrases = [
            "I hope this email finds you well",
            "As per our previous conversation",
            "Please find attached",
            "At your earliest convenience",
            "I wanted to reach out",
        ]
        
        # Urgency patterns common in AI phishing
        self.urgency_patterns = [
            r"urgent.*action.*required",
            r"immediate.*attention",
            r"within.*\d+.*hours",
            r"account.*suspend",
        ]
    
    def analyze(self, email_text: str) -> PhishingAnalysis:
        """Analyze email for AI-generated phishing indicators."""
        indicators = []
        scores = []
        
        # Check for perfect grammar
        score, indicator = self._check_perfect_grammar(email_text)
        if indicator:
            indicators.append(indicator)
        scores.append(score)
        
        # Check for urgency patterns
        score, indicator = self._check_urgency(email_text)
        if indicator:
            indicators.append(indicator)
        scores.append(score)
        
        # Check for template patterns
        score, indicator = self._check_templates(email_text)
        if indicator:
            indicators.append(indicator)
        scores.append(score)
        
        ai_probability = sum(scores) / len(scores) if scores else 0.0
        confidence = "high" if ai_probability > 0.7 else "medium" if ai_probability > 0.4 else "low"
        
        return PhishingAnalysis(
            text=email_text[:200] + "..." if len(email_text) > 200 else email_text,
            ai_probability=ai_probability,
            indicators=indicators,
            confidence=confidence,
            recommendations=self._get_recommendations(ai_probability),
        )
    
    def _check_perfect_grammar(self, text: str) -> tuple:
        human_errors = [r"\bi\b", r"\.{2}", r"\s{2,}"]
        error_count = sum(1 for p in human_errors if re.search(p, text))
        if len(text) > 500 and error_count == 0:
            return 0.7, "Suspiciously perfect grammar"
        return 0.2, None
    
    def _check_urgency(self, text: str) -> tuple:
        matches = sum(1 for p in self.urgency_patterns if re.search(p, text.lower()))
        if matches >= 2:
            return 0.7, f"Generic urgency patterns ({matches} found)"
        return 0.2, None
    
    def _check_templates(self, text: str) -> tuple:
        matches = sum(1 for phrase in self.ai_phrases if phrase.lower() in text.lower())
        if matches >= 2:
            return 0.6, f"AI-common phrases ({matches} found)"
        return 0.2, None
    
    def _get_recommendations(self, probability: float) -> List[str]:
        if probability > 0.7:
            return [
                "HIGH RISK: Treat as likely AI-generated phishing",
                "Do NOT click any links or download attachments",
                "Verify sender through out-of-band communication",
            ]
        elif probability > 0.4:
            return ["MEDIUM RISK: Exercise caution", "Verify sender identity"]
        return ["LOW RISK: Standard vigilance recommended"]

In [ ]:
# Test the detector
detector = AIPhishingDetector()

suspicious_email = """
Dear Valued Customer,

I hope this email finds you well. I wanted to reach out regarding your account.

Urgent action is required within 24 hours to verify your credentials.
Failure to respond will result in account suspension.

Please find attached the verification form.

Thank you for your prompt attention to this matter.

Best regards,
IT Security Team
"""

result = detector.analyze(suspicious_email)
print(f"AI Probability: {result.ai_probability:.1%}")
print(f"Confidence: {result.confidence}")
print(f"\nIndicators:")
for ind in result.indicators:
    print(f"  • {ind}")
print(f"\nRecommendations:")
for rec in result.recommendations:
    print(f"  • {rec}")

## Part 3: Voice Cloning Defense (Vishing)

### Scattered Spider Attack Pattern

```
┌─────────────────────────────────────────────────────────────────────────────┐
│              SCATTERED SPIDER VISHING ATTACK PATTERN                        │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  1. RECONNAISSANCE                                                          │
│     • Scrape LinkedIn for IT helpdesk staff names                          │
│     • Identify new employees (easier targets)                               │
│                                                                             │
│  2. VOICE CLONING PREPARATION                                              │
│     • Obtain voice samples from earnings calls, YouTube                    │
│     • Generate synthetic voice of known executive                           │
│                                                                             │
│  3. VISHING ATTACK                                                          │
│     • Call helpdesk claiming to be executive                               │
│     • Request MFA reset or password reset                                   │
│     • Use urgency: "I'm about to board a flight"                           │
│                                                                             │
│  DEFENSE: Out-of-band verification, code words, security awareness         │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
class VishingDetector:
    """Framework for detecting AI-powered vishing attacks."""
    
    def __init__(self):
        self.high_risk_scenarios = [
            "wire_transfer_request",
            "credential_request",
            "mfa_bypass_request",
            "emergency_access_request",
        ]
        
        self.challenge_questions = [
            "What did we discuss in our last meeting?",
            "Can you remind me of the project we worked on together?",
            "What floor is your office on?",
        ]
    
    def analyze_call(self, request_type: str, urgency: str, 
                     callback_offered: bool, verification_accepted: bool) -> dict:
        """Analyze call context for vishing indicators."""
        risk_score = 0.0
        
        if request_type in self.high_risk_scenarios:
            risk_score += 0.3
        if urgency in ["critical", "emergency"]:
            risk_score += 0.25
        if not callback_offered:
            risk_score += 0.2
        if not verification_accepted:
            risk_score += 0.25
        
        if risk_score > 0.6:
            recommendation = "⚠️ HIGH RISK: Likely vishing attempt"
        elif risk_score > 0.3:
            recommendation = "⚡ ELEVATED RISK: Proceed with caution"
        else:
            recommendation = "✅ LOWER RISK: Follow standard procedures"
        
        return {
            "risk_score": risk_score,
            "recommendation": recommendation,
            "verification_protocol": self.get_verification_protocol()
        }
    
    def get_verification_protocol(self) -> List[str]:
        return [
            "1. Tell caller you need to verify - legitimate callers expect this",
            "2. Get a callback number and verify it independently",
            "3. Call back using a known-good number from company directory",
            "4. Use a pre-established code word if available",
        ]

# Test
vishing_detector = VishingDetector()
result = vishing_detector.analyze_call(
    request_type="mfa_bypass_request",
    urgency="emergency",
    callback_offered=False,
    verification_accepted=False
)
print(f"Risk Score: {result['risk_score']:.0%}")
print(f"Recommendation: {result['recommendation']}")

## Part 4: AI-Enhanced Malware Analysis

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                 AI-ENHANCED MALWARE: DEFENDER'S GUIDE                       │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  OFFENSIVE AI CAPABILITIES         │  DEFENSIVE COUNTERMEASURES            │
│  ──────────────────────────────    │  ───────────────────────────────      │
│                                    │                                        │
│  • Polymorphic code generation     │  • Behavioral analysis                 │
│  • Automated vulnerability         │  • Aggressive patching                 │
│    discovery & exploitation        │                                        │
│  • Evasion testing at scale        │  • Defense-in-depth                    │
│  • Social engineering at scale     │  • AI-powered email filtering          │
│                                    │                                        │
│  KEY INSIGHT: Traditional signatures are losing effectiveness.              │
│  The future of detection is behavioral and AI-assisted.                    │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
class AIEnhancement(Enum):
    POLYMORPHIC_CODE = "polymorphic_code"
    EVASION_OPTIMIZATION = "evasion_optimization"
    SOCIAL_ENGINEERING = "social_engineering"


class AIMalwareAnalyzer:
    """Analyze malware for AI-assisted development indicators."""
    
    def get_detection_strategies(self, enhancement_type: AIEnhancement) -> List[str]:
        strategies = {
            AIEnhancement.POLYMORPHIC_CODE: [
                "Focus on behavioral detection over signatures",
                "Monitor for code generation API calls",
                "Use ML-based detection that generalizes across variants",
            ],
            AIEnhancement.EVASION_OPTIMIZATION: [
                "Implement defense-in-depth (multiple detection layers)",
                "Use canary/honeypot techniques",
                "Deploy behavioral analysis in sandboxes",
            ],
            AIEnhancement.SOCIAL_ENGINEERING: [
                "Train users on AI-generated content indicators",
                "Implement email authentication (DMARC, DKIM, SPF)",
                "Use AI-based phishing detection (fight AI with AI)",
            ],
        }
        return strategies.get(enhancement_type, ["Monitor for unusual patterns"])

# Example usage
analyzer = AIMalwareAnalyzer()
print("Detection strategies for polymorphic code:")
for strategy in analyzer.get_detection_strategies(AIEnhancement.POLYMORPHIC_CODE):
    print(f"  • {strategy}")

## Key Takeaways

1. **AI-Generated Content Detection** - Look for "too perfect" grammar and consistency
2. **Voice Cloning Defenses** - Always verify through out-of-band communication
3. **AI-Enhanced Malware** - Signature-based detection is losing effectiveness
4. **Fight AI with AI** - Use AI-powered defenses against AI-powered attacks

## Next Steps

- **Lab 17**: Adversarial ML - Attack and defend ML models
- **Lab 20**: LLM Red Teaming - Offensive security for LLM systems
- **Lab 20b**: AI-Assisted Purple Team - Collaborative AI exercises